### 1. 훈련 데이터셋으로 MCPAS.csv 이용, 검정 데이터셋으로 VDJDB.csv 이용 
### 2. 훈련 데이터셋으로 VDJDB.csv 이용, 검정 데이터셋으로 MCPAS.csv 이용
### 3. 1번, 2번 훈련할 때, ResNet과 Random Forest 이용해서 모델 훈련. 평가할 때, F1-Score, MCC, AUPRC 척도로 평가

##### 데이터셋 지정
  1. MCPAS.csv -> df_1
  2. VDJDB.csv -> df_2

In [39]:
import pandas as pd

df_1 = pd.read_csv('MCPAS.csv')

print(df_1)

       answer  model_2_ptm_plddt  model_2_ptm_pae  model_2_ptm_plddt_0  \
0           1          92.701679         6.750027            94.686831   
1           1          92.871003         7.036577            94.951055   
2           1          92.798729         7.374712            94.923411   
3           1          90.716556         9.961738            94.343239   
4           1          90.986426         9.003108            94.605061   
...       ...                ...              ...                  ...   
12469       0          91.763947         8.485618            94.097745   
12470       0          91.998154         8.395148            94.335725   
12471       0          91.702185         7.806988            94.199506   
12472       0          91.497231         8.119705            93.999229   
12473       0          91.945176         7.835033            94.450396   

       model_2_ptm_pae_0_0  model_2_ptm_pae_0_1  model_2_ptm_pae_0_2  \
0                 2.504075             

In [40]:
df_2 = pd.read_csv('VDJDB.csv')

print(df_2)

       answer  model_2_ptm_plddt  model_2_ptm_pae  model_2_ptm_plddt_0  \
0           1          94.389360         5.785817            94.758111   
1           1          94.142023         6.131019            94.452999   
2           1          94.079617         6.273674            94.474464   
3           1          94.851496         5.367987            95.582371   
4           1          93.479146         7.097644            94.251281   
...       ...                ...              ...                  ...   
10510       0          90.741382         8.106626            92.852356   
10511       0          90.809603         8.432978            92.905660   
10512       0          89.026760        11.148018            92.804020   
10513       0          89.026218        11.402861            92.774354   
10514       0          89.240778        11.189574            92.938956   

       model_2_ptm_pae_0_0  model_2_ptm_pae_0_1  model_2_ptm_pae_0_2  \
0                 2.678177             

# 1. 훈련 데이터셋 - MCPAS.csv, 검정 데이터셋  - VDJDB.csv

## ResNet 모델

### resnet 사용을 위해 2차원 df_1을 2차원 데이터로 변환

In [41]:
import numpy as np
import pandas as pd

# 첫 번째 열(출력 변수)을 제외한 나머지 열을 선택
train_labels = df_1.iloc[:, 0]  # 첫 번째 열 - 출력 변수 분리
train_input_data = df_1.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

train_input_data = train_input_data.values
train_input_data

array([[ 9.27016788e+01,  6.75002718e+00,  9.46868311e+01, ...,
         0.00000000e+00,  0.00000000e+00, -7.11000000e-15],
       [ 9.28710029e+01,  7.03657723e+00,  9.49510548e+01, ...,
         0.00000000e+00,  0.00000000e+00, -7.11000000e-15],
       [ 9.27987293e+01,  7.37471247e+00,  9.49234108e+01, ...,
         0.00000000e+00,  0.00000000e+00,  3.55000000e-15],
       ...,
       [ 9.17021851e+01,  7.80698824e+00,  9.41995059e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.14972312e+01,  8.11970520e+00,  9.39992286e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.19451756e+01,  7.83503342e+00,  9.44503963e+01, ...,
         0.00000000e+00,  0.00000000e+00, -1.00000000e-03]])

In [42]:
# 각 샘플을 10x10 형태로 재배열
reshaped_train_data = np.zeros((train_input_data.shape[0], 10, 10))

for i, sample in enumerate(train_input_data):
    padded_sample = np.pad(sample, (0, 3), 'constant')

    reshaped_sample = padded_sample.reshape(10, 10)

    reshaped_train_data[i] = reshaped_sample

print("변환된 데이터의 형태:", reshaped_train_data.shape)

train_data = reshaped_train_data

변환된 데이터의 형태: (12474, 10, 10)


In [43]:
train_data

array([[[ 9.27016788e+01,  6.75002718e+00,  9.46868311e+01, ...,
          8.90083720e+01,  3.08231688e+00,  1.52338004e+00],
        [ 1.03176479e+01,  1.03108768e+01,  9.19326121e+01, ...,
          9.06744624e+01,  1.16499462e+01,  9.02455711e+00],
        [ 5.83384848e+00,  3.62525821e+00, -1.02394800e+03, ...,
          4.01000000e+00,  7.59880000e+01,  2.94944000e+02],
        ...,
        [-3.29600000e+00, -9.96100000e+00, -1.34760000e+01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -2.99220000e+01,  1.00000000e-03, ...,
          0.00000000e+00,  1.00000000e-03, -8.78280000e+01],
        [-1.42000000e-14, -1.42000000e-14,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 9.28710029e+01,  7.03657723e+00,  9.49510548e+01, ...,
          8.94222025e+01,  3.01622653e+00,  1.49793160e+00],
        [ 1.05464973e+01,  1.10033646e+01,  9.16009102e+01, ...,
          9.11534601e+01,  1.21117430e

In [44]:
# 채널에 차원 추가

train_data = train_data.reshape(-1, 10, 10, 1)

print("채널이 추가된 데이터의 형태:", train_data.shape)

## 12474개의 10*10 크기 단일 채널 이미지 데이터

채널이 추가된 데이터의 형태: (12474, 10, 10, 1)


In [45]:
train_labels.shape

(12474,)

In [58]:
print(train_labels.unique())

[1 0]


In [46]:
train_labels

0        1
1        1
2        1
3        1
4        1
        ..
12469    0
12470    0
12471    0
12472    0
12473    0
Name: answer, Length: 12474, dtype: int64

### resnet 사용을 위해 2차원 df_2을 2차원 데이터로 변환

In [47]:
import numpy as np
import pandas as pd

# 첫 번째 열이 출력 변수
test_labels = df_2.iloc[:, 0]  
# 첫 번째 열(출력 변수)을 제외한 나머지 열을 선택
test_input_data = df_2.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

test_input_data = test_input_data.values
test_input_data

array([[ 9.43893598e+01,  5.78581715e+00,  9.47581111e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.41420226e+01,  6.13101911e+00,  9.44529988e+01, ...,
         0.00000000e+00,  0.00000000e+00, -3.55000000e-15],
       [ 9.40796174e+01,  6.27367449e+00,  9.44744640e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 8.90267602e+01,  1.11480179e+01,  9.28040202e+01, ...,
         0.00000000e+00,  0.00000000e+00, -5.33000000e-15],
       [ 8.90262184e+01,  1.14028606e+01,  9.27743542e+01, ...,
         0.00000000e+00,  0.00000000e+00, -8.88000000e-15],
       [ 8.92407784e+01,  1.11895742e+01,  9.29389556e+01, ...,
         0.00000000e+00,  0.00000000e+00,  5.33000000e-15]])

In [48]:
# 각 샘플을 10x10 형태로 재배열
reshaped_test_data = np.zeros((test_input_data.shape[0], 10, 10))

for i, sample in enumerate(test_input_data):
    padded_sample = np.pad(sample, (0, 3), 'constant')

    reshaped_sample = padded_sample.reshape(10, 10)

    reshaped_test_data[i] = reshaped_sample

print("변환된 데이터의 형태:", reshaped_test_data.shape)

test_data = reshaped_test_data


변환된 데이터의 형태: (10515, 10, 10)


In [49]:
test_data

array([[[ 9.43893598e+01,  5.78581715e+00,  9.47581111e+01, ...,
          7.93575946e+01,  8.84212303e+00,  3.75757718e+00],
        [ 1.50828104e+01,  1.45931930e+01,  9.44703725e+01, ...,
          9.50757419e+01,  8.50940323e+00,  6.63486338e+00],
        [ 3.82633877e+00,  2.37067437e+00, -1.05314600e+03, ...,
          3.92500000e+00,  7.54640000e+01,  2.79029000e+02],
        ...,
        [-2.97100000e+00, -5.90100000e+00, -9.68400000e+00, ...,
         -1.00000000e-03, -1.00500000e+00,  0.00000000e+00],
        [-1.00500000e+00, -2.94250000e+01, -2.97000000e-01, ...,
         -1.33000000e-01, -4.30000000e-01, -8.64400000e+01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 9.41420226e+01,  6.13101911e+00,  9.44529988e+01, ...,
          7.72284901e+01,  9.36628819e+00,  3.89607453e+00],
        [ 1.58449535e+01,  1.58514509e+01,  9.41280569e+01, ...,
          9.51799753e+01,  9.16736126e

In [50]:
# 채널 차원 추가
test_data = test_data.reshape(-1, 10, 10, 1)

print("채널이 추가된 데이터의 형태:", test_data.shape)

## 12474개의 10*10 크기 단일 채널 이미지 데이터

채널이 추가된 데이터의 형태: (10515, 10, 10, 1)


In [51]:
test_labels.shape

(10515,)

In [52]:
test_labels

0        1
1        1
2        1
3        1
4        1
        ..
10510    0
10511    0
10512    0
10513    0
10514    0
Name: answer, Length: 10515, dtype: int64

### 입력 채널, 출력 채널 수정(resnet50 모델 수정)

In [53]:
# ResNet50 기반, 10*10 단일 채널 입력, 단일 출력인 모델로 수정

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# Resnet50 모델 불러오기, 첫 번째 층과 마지막 층 제외
base_model = ResNet50(weights=None, include_top=False, input_tensor=Input(shape=(10, 10, 1)))

# Global Average Pooling 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 완전 연결 계층 추가(출력 변수 하나)
predictions = Dense(1, activation='linear')(x)

# 새로운 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 구조 확인
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 10, 10, 1)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 16, 16, 1)            0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 5, 5, 64)             3200      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 5, 5, 64)             256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [55]:
# 모양 확인
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

(12474, 10, 10, 1)
(12474,)
(10515, 10, 10, 1)
(10515,)


In [56]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

Epoch 1/10
390/390 [==============================] - 404s 1s/step - loss: 1.1581 - val_loss: 0.2465
Epoch 2/10
390/390 [==============================] - 398s 1s/step - loss: 0.2032 - val_loss: 0.1587
Epoch 3/10
390/390 [==============================] - 373s 956ms/step - loss: 0.1920 - val_loss: 0.2348
Epoch 4/10
390/390 [==============================] - 362s 927ms/step - loss: 0.1939 - val_loss: 3.2869
Epoch 5/10
390/390 [==============================] - 359s 921ms/step - loss: 0.1957 - val_loss: 3.4952
Epoch 6/10
390/390 [==============================] - 365s 935ms/step - loss: 0.1835 - val_loss: 3.3343
Epoch 7/10
390/390 [==============================] - 363s 932ms/step - loss: 0.1597 - val_loss: 4.3827
Epoch 8/10
390/390 [==============================] - 362s 929ms/step - loss: 0.1589 - val_loss: 15.4319
Epoch 9/10
390/390 [==============================] - 411s 1s/step - loss: 0.1648 - val_loss: 100.9431
Epoch 10/10
390/390 [==============================] - 385s 987ms/step

In [59]:
# 성능 평가 - f1, MCC, AUPRC

from sklearn.metrics import f1_score, matthews_corrcoef, average_precision_score

# 모델 예측
predictions = model.predict(test_data)

# 예측 결과를 이진 레이블로 변환
binary_predictions = (predictions > 0.5).astype(int)

# 실제 레이블
true_labels = test_labels # 테스트 데이터의 실제 레이블


329/329 [==============================] - 10s 27ms/step


In [60]:
# F1 계산
f1 = f1_score(true_labels, binary_predictions)

# MCC 계산
mcc = matthews_corrcoef(true_labels, binary_predictions)

# AUPRC 계산
auprc = average_precision_score(true_labels, predictions)

print("F1-score:", f1)
print("MCC:", mcc)
print("AUPRC:", auprc)

F1-score: 0.25099800399201594
MCC: 0.0
AUPRC: 0.19553165551109708


### 모델 성능 평가 해석
#### F1-score: 0.25099800399201594 
=> 0에 가깝기 때문에, 모델의 성능이 낮음. 정밀도와 재현율 둘 다 낮은 수치

#### MCC: 0.0
=> 모델의 성능이 무작위 추측과 동일하다는 것을 의미

#### AUPRC: 0.19553165551109708
=> 정밀도-재현율 곡선 아래의 면적을 나타냄. 0에 가까운 수치이기 때문에, 모델의 성능이 낮다는 것을 의미. 불균형한 데이터셋에서는 AUPRC가 모델 성능을 평가하는 데 중요한 지표.

=> 총평 : ResNet 모델이 데이터셋에 대해 낮은 성능을 보임. 데이터의 특성이나 모델의 구조가 적합하지 않음을 알 수 있음. 

## 랜덤포레스트 모델

In [89]:
from sklearn.ensemble import RandomForestClassifier  # 분류 문제인 경우
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # 분류 문제인 경우

# 데이터 준비
X = df_1.iloc[:, 1:]  # 입력 변수
y = df_1.iloc[:, 0]   # 목표 변수

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [65]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9979, 97)
(2495, 97)
(9979,)
(2495,)


In [66]:
# 랜덤 포레스트 모델 구성
model = RandomForestClassifier()

# 모델 학습
model.fit(X_train, y_train)

RandomForestClassifier()

In [67]:
from sklearn.metrics import f1_score, matthews_corrcoef, average_precision_score, precision_recall_curve, auc

# 모델 성능 평가 - 예측
predictions = model.predict(X_test)

# 예측 확률 계산 (AUPRC 평가)
probabilities = model.predict_proba(X_test)[:, 1]

# F1-score 계산
f1 = f1_score(y_test, predictions)

# MCC
mcc = matthews_corrcoef(y_test, predictions)

# PRC, AUPRC
precision, recall, _ = precision_recall_curve(y_test, probabilities)
auprc = auc(recall, precision)

print("F1-score:", f1)
print("MCC:", mcc)
print("AUPRC:", auprc)

F1-score: 0.7341772151898734
MCC: 0.7098889278753726
AUPRC: 0.9355262415435939


### 모델 성능 평가 해석
#### F1-score: 0.7341772151898734
=> 모델이 상당히 균형 잡힌 정밀도와 재현율을 가지고 있음. 모델이 클래스를 잘 구별하며, 불균형 데이터셋에서 좋은 성능을 보임

#### MCC: 0.7098889278753726
=> 모델이 양성 및 음성 클래스를 모두 잘 구별. 모든 분류 범주에서 균형 잡힌 성능을 보임

#### AUPRC: 0.9355262415435939
=> 모델이 1의 클래스에 대해 높은 성능을 보임. 불균형한 데이터셋에서 중요.

총평 => 랜덤 포레스트 모델은 제시된 데이터셋에 대해 높은 성능을 보이고 있음을 알 수 있음. MCC와 AUPRC 값이 높은 것은, 모델이 데이터셋의 양성 및 음성 클래스를 모두 잘 예측함을 알 수 있음. F1-score도 높아서, 균형 잡힌 정밀도와 재현율을 가지고 있음을 알 수 있음.

# 2. 훈련 데이터셋 VDJDB.csv(df_2), 검정 데이터셋 MCPAS.csv(df_1)

## ResNet 모델

### ResNet 사용을 위해 2차원 df_2를 2차원 데이터로 변환

In [ ]:
import numpy as np
import pandas as pd

# 첫 번째 열(출력 변수)을 제외한 나머지 열을 선택
train_labels = df_1.iloc[:, 0]  # 첫 번째 열 - 출력 변수 분리
train_input_data = df_1.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

train_input_data = train_input_data.values
train_input_data

정리

1. df_1
첫번째 열(출력 변수) : train_labels
나머지 열(입력 변수): train_data

2. df_2
첫 번쨰 열(출력 변수) : test_labels
나머지 열(입력 변수) : test_data

In [71]:
print(train_labels.shape)
print(train_data.shape)
print(test_labels.shape)
print(test_data.shape)

(12474,)
(12474, 10, 10, 1)
(10515,)
(10515, 10, 10, 1)


### train 데이터와 test 데이터 바꿔주기

In [72]:
# 기존 훈련 데이터와 테스트 데이터의 이름을 서로 교환
new_train_data = test_data
new_train_labels = test_labels
new_test_data = train_data
new_test_labels = train_labels

In [73]:
print(new_train_labels.shape)
print(new_train_data.shape)
print(new_test_labels.shape)
print(new_test_data.shape)

(10515,)
(10515, 10, 10, 1)
(12474,)
(12474, 10, 10, 1)


## 입력 채널, 출력 채널 수정

In [74]:
# ResNet50 기반, 10*10 단일 채널 입력, 단일 출력인 모델로 수정

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# Resnet50 모델 불러오기, 첫 번째 층과 마지막 층 제외
base_model = ResNet50(weights=None, include_top=False, input_tensor=Input(shape=(10, 10, 1)))

# Global Average Pooling 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 완전 연결 계층 추가(출력 변수 하나)
predictions = Dense(1, activation='linear')(x)

# 새로운 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 구조 확인
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 10, 10, 1)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 16, 16, 1)            0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 5, 5, 64)             3200      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 5, 5, 64)             256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [75]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

Epoch 1/10
390/390 [==============================] - 394s 983ms/step - loss: 1.1572 - val_loss: 0.6861
Epoch 2/10
390/390 [==============================] - 379s 973ms/step - loss: 0.2183 - val_loss: 0.5078
Epoch 3/10
390/390 [==============================] - 411s 1s/step - loss: 0.1868 - val_loss: 0.1639
Epoch 4/10
390/390 [==============================] - 453s 1s/step - loss: 0.1732 - val_loss: 1.8611
Epoch 5/10
390/390 [==============================] - 436s 1s/step - loss: 0.1722 - val_loss: 0.1504
Epoch 6/10
390/390 [==============================] - 430s 1s/step - loss: 0.1797 - val_loss: 0.1445
Epoch 7/10
390/390 [==============================] - 453s 1s/step - loss: 0.1839 - val_loss: 76.7457
Epoch 8/10
390/390 [==============================] - 439s 1s/step - loss: 0.1568 - val_loss: 2.6280
Epoch 9/10
390/390 [==============================] - 436s 1s/step - loss: 0.1693 - val_loss: 0.8014
Epoch 10/10
390/390 [==============================] - 429s 1s/step - loss: 0.1849 -

In [78]:
# 성능 평가 - f1, MCC, AUPRC

from sklearn.metrics import f1_score, matthews_corrcoef, average_precision_score

# 모델 예측
predictions = model.predict(test_data)

# 예측 결과를 이진 레이블로 변환
binary_predictions = (predictions > 0.5).astype(int)

# 실제 레이블
true_labels = test_labels # 테스트 데이터의 실제 레이블

# F1-score 계산
f1 = f1_score(true_labels, binary_predictions)

# Matthews Correlation Coefficient 계산
mcc = matthews_corrcoef(true_labels, binary_predictions)

# AUPRC 계산
auprc = average_precision_score(true_labels, predictions)

print("F1-score:", f1)
print("MCC:", mcc)
print("AUPRC:", auprc)



329/329 [==============================] - 10s 30ms/step
F1-score: 0.25099800399201594
MCC: 0.0
AUPRC: 0.13692194185814158


### 모델 성능 평가 해석
#### F1-score: 0.25099800399201594
=> 0에 가깝기 때문에, 모델의 성능이 낮음. 정밀도와 재현율 둘 다 낮은 수치

#### MCC: 0.0
=> 모델의 성능이 무작위 추측과 동일하다는 것을 의미

#### AUPRC: 0.13692194185814158
=> 정밀도-재현율 곡선 아래의 면적을 나타냄. 0에 가까운 수치이기 때문에, 모델의 성능이 낮다는 것을 의미. 불균형한 데이터셋에서는 AUPRC가 모델 성능을 평가하는 데 중요한 지표.

=> 총평 : ResNet 모델이 데이터셋에 대해 낮은 성능을 보임. 데이터의 특성이나 모델의 구조가 적합하지 않음을 알 수 있음. 

# 랜덤포레스트 모델

In [90]:
from sklearn.ensemble import RandomForestClassifier  # 분류 문제인 경우
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # 분류 문제인 경우

# 데이터 준비
X = df_2.iloc[:, 1:]  # 입력 변수
y = df_2.iloc[:, 0]   # 목표 변수

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [83]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8412, 97)
(2103, 97)
(8412,)
(2103,)


In [84]:
# 랜덤 포레스트 모델 구성
model = RandomForestClassifier()

# 모델 학습
model.fit(X_train, y_train)

RandomForestClassifier()

In [85]:
from sklearn.metrics import f1_score, matthews_corrcoef, average_precision_score, precision_recall_curve, auc

# 모델 성능 평가 - 예측
predictions = model.predict(X_test)

# 예측 확률 계산 (AUPRC 평가)
probabilities = model.predict_proba(X_test)[:, 1]

# F1-score 계산
f1 = f1_score(y_test, predictions)

# MCC
mcc = matthews_corrcoef(y_test, predictions)

# PRC, AUPRC
precision, recall, _ = precision_recall_curve(y_test, probabilities)
auprc = auc(recall, precision)

print("F1-score:", f1)
print("MCC:", mcc)
print("AUPRC:", auprc)

F1-score: 0.8171206225680934
MCC: 0.8030716571382289
AUPRC: 0.9362653584494931


### 모델 성능 평가 해석
#### F1-score: 0.8171206225680934
=> 모델이 상당히 균형 잡힌 정밀도와 재현율을 가지고 있음. 모델이 클래스를 잘 구별하며, 불균형 데이터셋에서 좋은 성능을 보임

#### MCC: 0.8030716571382289
=> 모델이 양성 및 음성 클래스를 모두 잘 구별. 모든 분류 범주에서 균형 잡힌 성능을 보임

#### AUPRC: 0.9362653584494931
=> 모델이 1의 클래스에 대해 높은 성능을 보임. 불균형한 데이터셋에서 중요.

총평 => 랜덤 포레스트 모델은 제시된 데이터셋에 대해 높은 성능을 보이고 있음을 알 수 있음. MCC와 AUPRC 값이 높은 것은, 모델이 데이터셋의 양성 및 음성 클래스를 모두 잘 예측함을 알 수 있음. F1-score도 높아서, 균형 잡힌 정밀도와 재현율을 가지고 있음을 알 수 있음.